In [1]:
import requests
import os
from dotenv import load_dotenv
import json
import pandas as pd
import pymysql
import numpy as np
from tqdm import tqdm

In [2]:
load_dotenv()

True

In [3]:
GEO_LOCATION_API_KEY = os.getenv('GEOLOCATION_API_KEY')

In [4]:
request_url = f'https://api.tomtom.com/search/2/poiSearch/restaurant.json?key={GEO_LOCATION_API_KEY}&limit=1000&lat=-23.576604&lon=-46.668532&radius=100000000'

In [5]:
def request_given_lat_lon(lat, lon):
    url = f'https://api.tomtom.com/search/2/poiSearch/restaurant.json?key={GEO_LOCATION_API_KEY}&limit=100&lat={lat}&lon={lon}&radius=1000'
    response = requests.get(request_url)
    decoded_response = json.loads(response.content)
    return decoded_response['results']

In [6]:
restaurant_list = []

In [7]:
lat, lon = -23.697907, -46.828054
final_lat, final_lon = -23.506020, -46.419272

lat_range = np.arange(lat, final_lat, 0.01)
lon_range = np.arange(lon, final_lon, 0.01)


In [8]:
lat_range.shape[0]

20

In [9]:
lon_range.shape[0]

41

In [10]:

for la in tqdm(range(len(lat_range))):
    for lo in lon_range:
        restaurant_list.extend(request_given_lat_lon(la, lo))

 30%|███       | 6/20 [02:03<04:44, 20.29s/it]

In [ ]:
restaurant_info = {
    'name' : [],
    'lat' : [],
    'lon' : [] ,
    'street_name' : [],
    'neighborhood': []
}
for restaurant in restaurant_list:
    if 'name' in restaurant['poi'] and 'lat' in restaurant['position'] and\
         'lon' in restaurant['position'] and 'streetName' in restaurant['address'] and\
         'municipalitySubdivision' in restaurant['address']:
        restaurant_info['name'].append(restaurant['poi']['name'])
        restaurant_info['lat'].append(restaurant['position']['lat'])
        restaurant_info['lon'].append(restaurant['position']['lon'])
        restaurant_info['street_name'].append(restaurant['address']['streetName'])
        restaurant_info['neighborhood'].append(restaurant['address']['municipalitySubdivision'])
    

In [ ]:
df = pd.DataFrame(restaurant_info)

In [ ]:
df.shape

(53538, 5)

In [ ]:
df.head()

,name,lat,lon,street_name,neighborhood
0,E Sports Burguer,-23.57711,-46.66889,Rua General Fonseca Teles,Jardim Paulista
1,Mariolão Lanches,-23.57665,-46.67052,Avenida Nove de Julho,Jardim Paulista
2,Mj Lanches,-23.57475,-46.66683,Rua Maestro Elias Lobo,Jardim Paulista
3,Karmines Alimentícios,-23.57840,-46.67051,Rua Susano,Jardim Paulista
4,Cafelito,-23.57594,-46.67140,Rua Groelândia,Jardim Paulista


In [ ]:
QUERY_BEGINNING = 'INSERT INTO restaurants VALUES '

In [ ]:
for index, row in df.iterrows():
    QUERY_BEGINNING += f'''(NULL, "{row['name']}", {row['lat']}, {row['lon']}, "{row['street_name']}", "{row['neighborhood']}"), \n'''

QUERY_BEGINNING = QUERY_BEGINNING[:-3] + ';'

In [ ]:
print(QUERY_BEGINNING)

INSERT INTO restaurants VALUES (NULL, "E Sports Burguer", -23.57711, -46.66889, "Rua General Fonseca Teles", "Jardim Paulista"), 
(NULL, "Mariolão Lanches", -23.57665, -46.67052, "Avenida Nove de Julho", "Jardim Paulista"), 
(NULL, "Mj Lanches", -23.57475, -46.66683, "Rua Maestro Elias Lobo", "Jardim Paulista"), 
(NULL, "Karmines Alimentícios", -23.5784, -46.67051, "Rua Susano", "Jardim Paulista"), 
(NULL, "Cafelito", -23.57594, -46.6714, "Rua Groelândia", "Jardim Paulista"), 
(NULL, "Bar e Empório Pires", -23.57936, -46.6681, "Rua Primavera", "Jardim Paulista"), 
(NULL, "Lanchonete e Restaurante Castelo", -23.57844, -46.67081, "Avenida São Gabriel", "Jardim Paulista"), 
(NULL, "Tutti Lanches", -23.57721, -46.67174, "Avenida Nove de Julho", "Itaim Bibi"), 
(NULL, "Granito Lanches", -23.57827, -46.67127, "Avenida São Gabriel", "Itaim Bibi"), 
(NULL, "Olaria Acessórios de Moda", -23.57828, -46.67132, "Avenida São Gabriel", "Itaim Bibi"), 
(NULL, "DMPB Comércio de Alimentos", -23.57828, -

In [ ]:
db_host = os.getenv('HOSTNAME')
db_user = os.getenv('HOST')
db_password = os.getenv('PASSWORD')
db_name = os.getenv('DB_NAME')

In [ ]:

connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    db=db_name
)

In [ ]:
connection.ping()

In [ ]:
cursor = pymysql.cursors.Cursor(connection)

In [ ]:
cursor.execute('TRUNCATE TABLE restaurants')

In [ ]:
cursor.execute(QUERY_BEGINNING)

32